In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_excel("Manually_Tagged_Sheet.xlsx",sheet_name="Valid Entries_updated_15_06_202")
# data.head()
data=data.drop(columns=["exact_matched_patt","spacy_format_v1","spacy_format_v1","summaries_matching"])
data.columns=["id","summaries","exact_matched_patt_contextual","Tech_from_string_match"]

In [3]:
data=data.loc[:2500]

In [4]:
data=data.sort_values(by=['id'])
data=data.reset_index(drop=True)
len(data)

2501

In [5]:
data1=data.loc[:499]
data2=data.loc[500:999]
data3=data.loc[1000:1499]
data4=data.loc[1500:1999]
data5=data.loc[2000:2499]

In [6]:
len(data1),len(data2),len(data3),len(data4),len(data5)

(500, 500, 500, 500, 500)

In [7]:
data=data5 ##########

In [8]:
data=data.reset_index(drop=True)

In [9]:
data.to_csv("SPLIT5_DATA_FOR_PREDICTION.csv") ##########

In [10]:
#replacing null vals with empty list
data['exact_matched_patt_contextual'] = [ [] if x is np.NaN else x for x in data['exact_matched_patt_contextual'] ]

import nltk
import re
from bs4 import BeautifulSoup

nltk.download("punkt")

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mohitbagaria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# for i, l in enumerate(data["exact_matched_patt_contextual"]):
#  print("list",i,"is",type(l))

In [12]:
data['exact_matched_patt_contextual']=data['exact_matched_patt_contextual'].astype(str)

In [13]:
# for i, l in enumerate(data["exact_matched_patt_contextual"]):
#  print("list",i,"is",type(l))

In [14]:
# data.head(15)

In [15]:
def clean_alt_list(list_):
#     list_ = list_.replace(', ', "','")
#     list_ = list_.replace('[', "['")
#     list_ = list_.replace(']', "']")
    
    if(list_[0]!='['):
        list_='['+list_
    if(list_[len(list_)-1]!=']'):
        list_+=']'
    return list_

In [16]:
data["exact_matched_patt_contextual"] = data["exact_matched_patt_contextual"].apply(clean_alt_list)

data["exact_matched_patt_contextual"] = data["exact_matched_patt_contextual"].apply(eval)

In [17]:
def give_tags(manual_tags): #a list
    tech_dict = {}
    for j in manual_tags:
        if j not in tech_dict:
            tech_dict[j] = 1
        else:
            tech_dict[j] += 1

    # tech_dict

    tech_list=[]
    tech_multi_words=[]
    for i in tech_dict:
        if ((' ' in i) == True):
            tech_multi_words.append(i)
        else:
            tech_list.append(i)
#     if(len(tech_multi_words)>0):
#         print(tech_multi_words)
    return tech_list,tech_multi_words

In [18]:
data["summaries"]=data["summaries"].apply(clean_text)
data.summaries = data.summaries.replace('\s+', ' ', regex=True)

In [19]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [20]:
def convert(lst):
    return ' '.join(lst).split()


def BIO_conversion(sentence,manual_tags):
    tech_list,tech_multi_words = give_tags(manual_tags)
    ind_dict = {}
    for i in tech_multi_words:
        if i in sentence:
#             ind_dict[sentence.index(i)]=i #word as val, key as index
            all_occ = list(find_all(sentence,i)) # [0, 5, 10, 15]
            for ind in all_occ:
                ind_dict[ind]=i

            
    temp_word=""
    final_word_sen=[]
    final_tag_sen = []
    start=-1
    end=-1
    for i in range(0, len(sentence)):
        if(i in range(start,end)):
#             print("break hua")
            continue
#         print(i)
        if sentence[i].isspace()==True and len(temp_word)>0:
            final_word_sen.append(temp_word)
            if temp_word in tech_list:
#                 print(temp_word)
                final_tag_sen.append("B")
            else:
                final_tag_sen.append("O")
            temp_word=""
        else: #not space
            try: #do we have multiword at this index
                temp_word=ind_dict[i]
#                 print("****")
                start=i+1
                end=i+len(temp_word) #here tag these multiwords and update new i
#                 print(i)
#                 print("****")
#                 print(temp_word)
                listofwords = convert([temp_word])
                f=True
                for word in listofwords:
                    final_word_sen.append(word)
                    if f:
                        final_tag_sen.append("B")
                        f=False
                    else:
                        final_tag_sen.append("I")
                temp_word=""
            except:
                if(sentence[i]!=' '):
                    temp_word+=sentence[i]
                
    return pd.DataFrame(list(zip(final_word_sen, final_tag_sen)),columns =['Word', 'Tag'])

In [21]:
final_data=pd.DataFrame()
split=2000 ##########
for i in range(0,len(data)):
    temp = BIO_conversion(data["summaries"][i],data["exact_matched_patt_contextual"][i])
    length = len(temp)
    wordd="Sentence :"+str(i+1+split) #sentence no.
    a=[wordd]*length
    temp.insert(0,"Sentence #",a)
    final_data = final_data.append(temp, ignore_index=True) #appending sentences in the required format

In [22]:
final_data['Tag'].value_counts()

O    97417
B     2591
I      568
Name: Tag, dtype: int64

In [23]:
df=final_data
df.loc[(df['Tag'] == 'B'), 'Tag'] = 'B-ORG'
df.loc[(df['Tag'] == 'I'), 'Tag'] = 'I-ORG'
df.to_csv("SPLIT5_BIO_tagged_data.csv",index=False)